# Bibliotecas Usadas

In [1]:
import random
import copy
import math
import numpy as np
from numpy.random import choice
from threading import Thread

from tqdm import tqdm_notebook as tqdm

<h1 style="color:#fff;background-color:#000;padding:30px;margin:0;border:0;text-align:center">JOGO</h1>

Uma classe para simular um jogo de xadrez apenas com rainhas

In [2]:

class Jogo:
    
    """
    Construtor da classe
    
    @param dimensao_tabuleiro: inteiro positivo para dimensao do tabuleiro quadrado
    """
    def __init__(self, dimensao_tabuleiro):
                
        #guardando a dimensao
        self._dimensao = dimensao_tabuleiro
        
        #inicializando o dicionario de pecas
        self._pos_damas = dict()
    
    
    """
    Metodo get da dimensao, disparado quando "self.dimensao" eh executado
    
    @return dimensao: a dimensao do tabuleiro quadrado
    """
    @property
    def dimensao(self): return self._dimensao

    """
    Metodo set da dimensao, disparado quando "self.dimensao = value" eh executado
    por medidas de consistencia, as pecas tem suas posicoes anuladas, porem as quantidades sao as mesmas
    
    @param value: devera ser um inteiro positivo
    """
    @dimensao.setter
    def dimensao(self, value):
         
        #verificacao de consistencia da entrada
        if isinstance(value, int) and value >= 0: 
            self._dimensao = value
            
            #zerando os valores das pecas
            for chave in self._pos_damas: self._pos_damas[chave] = None
            
        else: print("tipo {} nao aceito".format(type(value)))
    
    """
    Metodo get do pos_damas, disparado quando "self.pos_damas" eh executado
    
    @return pos_damas: o dicionario contendo todas as pecas e suas posicoes
    """
    @property
    def pos_damas(self): return self._pos_damas.copy()

    """
    Metodo set do pos_damas, disparado quando "self.pos_damas = value" eh executado
    
    @param value: devera ser um dicionario do tipo: chave --> tuple(x, y) tal que
    tanto x quanto y sejam inteiros dentro do intervalo [0, self.dimensao[
    """
    @pos_damas.setter
    def pos_damas(self, value):
        
        #verificacao de consistencia da entrada
        check = (isinstance(value, dict) and 
                 all(isinstance(e, tuple)
                     and len(e) == 2
                     and all(isinstance(i,int) and
                             i>=0 and i<self.dimensao
                             for i in e)
                     for e in value.values()))
        
        #registrando entrada
        if check: self._pos_damas = value
        
        else: print("{} nao aceito como pos_damas".format(value))
    
    """
    Metodo get para uma peca do tabuleiro, disparado quando "self[idx]" for executado
    
    @return pos_damas[idx]: um tuple(int, int) se a chave existir, None c.c.
    """
    def __getitem__(self, idx): return self._pos_damas.get(idx)

    """
    Metodo set para uma nova peca do tabuleiro ou atualizacao de existente,
    disparado quando "self[idx] = value" for executado
    
    @param idx: novo/existente peca que sera manipulada
    
    @param value: novo valor que sera inserido, devera ser um tuple(x, y) tal que
    tanto x quanto y sejam inteiros dentro do intervalo [0, self.dimensao[
    """
    def __setitem__(self, idx, value):
        
        #verificacao de consistencia
        check = (isinstance(value, tuple)
                 and len(value) == 2 and 
                 all(isinstance(i,int) and
                     i>=0 and
                     i<self.dimensao for i in value)) 
        
        #registrando entrada
        if check:
            self._pos_damas[idx] = value
        else: print("{} nao aceito como valor".format(value))
            
    """
    Metodo para descobrir as jogadas obvias (horizontais e verticais)
    
    @param idx: peca que sera movimentada
    
    @return jogadas: lista de posicoes validas para uma jogada
    """
    def getJogadasObvias(self, idx):
        #obtendo a posicao da peca que sera movimentada
        ponto = self[idx]
        
        #se nao existir, vida que segue
        if not ponto:
            print("peca {} nao inicializada".format(idx))
            return None
        
        #varrendo posicoes nos sentidos
        cima = [(ponto[0], i) for i in range(ponto[1]+1, self.dimensao)]
        
        baixo = [(ponto[0], i) for i in range(ponto[1]-1, -1, -1)]
        
        direita = [(i, ponto[1]) for i in range(ponto[0]+1, self.dimensao)]
        
        esquerda = [(i, ponto[1]) for i in range(ponto[0]-1, -1, -1)]
        
        #concatenando tudo
        return [*cima,
               *baixo,
               *direita,
               *esquerda]
    
    """
    Metodo para descobrir as jogadas diagonais 
    
    @param idx: peca que sera movimentada
    
    @return jogadas: lista de posicoes validas para uma jogada
    """
    def getJogadasDiagonais(self, idx):
        
        #obtendo a posicao da peca que sera movimentada
        ponto = self[idx]
        
        #se nao existir, vida que segue
        if not ponto:
            print("peca {} nao inicializada".format(idx))
            return None

        #varrendo posicoes nos sentidos
        cima_direita = [(ponto[0]+i, ponto[1]+i) for i in range(1, self.dimensao) if ponto[0]+i < self.dimensao and ponto[1]+i < self.dimensao]
        
        baixo_direita = [(ponto[0]+i, ponto[1]-i) for i in range(1, self.dimensao) if ponto[0]+i < self.dimensao and ponto[1]-i >=0 ]
        
        baixo_esquerda = [(ponto[0]-i, ponto[1]-i) for i in range(1, self.dimensao) if ponto[0]-i >= 0 and ponto[1]-i >= 0]
        
        cima_esquerda = [(ponto[0]-i, ponto[1]+i) for i in range(1, self.dimensao) if ponto[0]-i >= 0 and ponto[1]+i < self.dimensao]
        
        #concatenando tudo
        return [*cima_direita,
               *baixo_direita,
               *baixo_esquerda,
               *cima_esquerda]
    
    """
    Metodo para descobrir todas as jogadas possiveis 
    
    @param idx: peca que sera movimentada
    
    @return jogadas: lista de posicoes validas para uma jogada
    """
    def getTodasJogadas(self, idx):
        
        #obtendo a peca que sera movimentada
        ponto = self[idx]
        
        #se nao tiver ponto, vida que segue
        if not ponto:
            print("peca {} nao inicializada".format(idx))
            return None

        #coletando e concatenando as jogadas
        return [*self.getJogadasDiagonais(idx), *self.getJogadasObvias(idx)]
        
    def getEspacosLetais(self):
        
        casas_letais = set()
        for peca in self.pos_damas:
            casas_letais = casas_letais.union(self.getTodasJogadas(peca))
        return list(casas_letais)
        
    def getEspacosLivres(self):
        
        tabuleiro = set( (x,y) for x in range(0,self.dimensao) for y in range(0,self.dimensao) )
        
        casas_letais = set(self.pos_damas.values())
        #casas_letais = set()
        
        for peca in self.pos_damas:
            
            casas_letais = casas_letais.union(self.getTodasJogadas(peca))
        
        return list(tabuleiro - casas_letais)
    
    def aux_print(self, index):
        string = ""
        inicio = "├"
        meio = "┼"
        fim = "┤"
        if index == 0:
            inicio = '┌'
            meio = '┬'
            fim = '┐'
        if index == self.dimensao:
            inicio = '└'
            meio = '┴'
            fim = '┘'
        
        for i in range(self.dimensao):
            if i == 0:
                string += inicio + '───'
            elif i == self.dimensao-1:
                string += meio + '───' + fim
            else:
                string += meio + '───'
        return string + '\n'
                
    
    def __str__(self):
        '''
        ┌───┬───┬───┐
        │   │   │   │
        ├───┼───┼───┤
        │   │   │   │
        ├───┼───┼───┤
        │   │   │ ♛ │
        └───┴───┴───┘
        '''
        string = "\n"

        if self.dimensao <= 28:            
            pos = self.pos_damas.values()
            for i in range(self.dimensao):
                string += self.aux_print(i)
                string += '│'
                for j in range(self.dimensao):
                    if (i, j) in pos:
                        string += " ♛ │"
                    else: 
                        string += "   │"

                string += "\n"

            string += self.aux_print(i+1)
        else:
            pos = self.pos_damas.values()
            for i in range(self.dimensao):
                for j in range(self.dimensao):
                    if (i, j) in pos:
                        string += "♛ "
                    else: 
                        string += "□ "
                string += "\n"
        return string

        

<h1 style="color:#fff;background-color:#000;padding:30px;margin:0;border:0;text-align:center">SIMULATED AN...</h1>

In [3]:
def tabuleiroInincial(dimensao):
    jogo = Jogo(dimensao)
    jogo.pos_damas = {i: (i,i) for i in range(0, jogo.dimensao)}
    return jogo

In [4]:
def heuristicaOtimizada(jogo):
    
    #peso do tabuleiro
    peso = 0

    #conjunto de pecas nao verificadas
    partes_ocupadas = list(jogo.pos_damas.values())
    
    partes_letais = jogo.getEspacosLetais()
    
    casas_mortas = filter(lambda x: x in partes_letais, partes_ocupadas)
    
    return len(list(casas_mortas))

In [5]:
def heuristicaPadrao(jogo):
    
    #peso do tabuleiro
    peso = 0

    #conjunto de pecas nao verificadas
    pecas_nao_verificadas = list(jogo.pos_damas.keys())

    #enquanto tiver pecas nao verificadas faca:
    for peca in pecas_nao_verificadas:

        #tirar uma peca arbitraria
        #peca = pecas_nao_verificadas.pop()

        #coletar todas as possiveis jogadas dela
        casas_letais = jogo.getTodasJogadas(peca)

        #para as outras pecas
        for chave, valor in jogo.pos_damas.items():

            #verificar se elas estao entre as possiveis jogadas
            if valor in casas_letais:

                #incrementa o peso
                peso += 1

    return peso

In [6]:
def movimentacaoRandomica(jogo):
    
    novo_jogo = copy.deepcopy(jogo)
    
    #peca aleatoria
    peca = random.choice(list(novo_jogo.pos_damas.keys()))
    
    pos = novo_jogo[peca]
    
    #todos os movimentos verticais e horizontais
    movimentos = novo_jogo.getJogadasObvias(peca)
    movimentos_simples_verticais = list(filter(lambda x: x[0] == pos[0], movimentos))
    
    escolha = random.choice(movimentos_simples_verticais)
    
    novo_jogo[peca] = escolha
    
    #novo_jogo.jogada_anterior = (peca, escolha)
    
    return novo_jogo

In [7]:
def simulatedAnnealing(funcao_peso, funcao_jogada, jogo, MAX = 10000, C=3):
    
    atual = jogo
    
    for t in tqdm(range(1, MAX)):
    
        T = C/math.sqrt(t)
        
        #if T == 0: return atual
        
        viz = funcao_jogada(atual)
        
        #print(viz)
        p_atual = funcao_peso(atual)
        p_viz = funcao_peso(viz)
        
        #if T<= 0.0005 or p_viz == 0: return viz
        
        if p_atual == 0:
            return atual
        
        if p_viz < p_atual:
            atual = viz
        
        else:
            probabilidade = math.exp( (p_atual - p_viz)/T )
            #print(probabilidade, funcao_peso(atual))
            atual = random.choices([viz, atual], weights= [probabilidade, 1-probabilidade])[0]
        
    return atual

In [8]:
%%time
teste = simulatedAnnealing(heuristicaPadrao, 
                           movimentacaoRandomica,
                          tabuleiroInincial(40))

KeyboardInterrupt: 

In [123]:
%%time
teste = simulatedAnnealing(heuristicaOtimizada, 
                           movimentacaoRandomica,
                          tabuleiroInincial(8))

CPU times: user 260 ms, sys: 41 µs, total: 260 ms
Wall time: 349 ms


In [124]:
print(teste)

┌───┬───┬───┬───┬───┬───┬───┬───┐
│   │ ♛ │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │   │ ♛ │
├───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │ ♛ │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│ ♛ │   │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │ ♛ │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │ ♛ │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │ ♛ │   │
├───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │ ♛ │   │   │   │   │
└───┴───┴───┴───┴───┴───┴───┴───┘



In [114]:
heuristicaPadrao(teste)

0

<p style="color:lime;background-color:#fc0fc0;padding:50px;margin:0;border:0;text-align:center;font-size:40px;font-weight:bold;font-family:URW Chancery L">DEMONSTRAÇÃO</p>

In [ ]:
random.sample(range(1,25),)


In [ ]:
def cruzamentoPadrao(pai, mae):
    novo_jogo = copy.deepcopy(pai)
    
    quantidade_mae = random.choice(range(1,mae.dimensao-1))
    
    #pecas = random.sample(list(novo_jogo.pos_damas.keys()), quantidade_mae)
    pecas = list(novo_jogo.pos_damas.keys())[quantidade_mae:]
    for peca in pecas:
        novo_jogo[peca] = mae[peca]
        
    return novo_jogo

    

In [ ]:
def mutacaoPadrao(individuo, mutabilidade):
    
    novo_jogo = copy.deepcopy(individuo)
    
    pecas = random.sample(list(novo_jogo.pos_damas.keys()), mutabilidade)
    
    for peca in pecas:
        
        movimentos = novo_jogo.getJogadasObvias(peca)
        pos = novo_jogo[peca]
        movimentos_simples_verticais = list(filter(lambda x: x[0] == pos[0], movimentos))

        escolha = random.choice(movimentos_simples_verticais)
        
        novo_jogo[peca] = escolha
        
    return novo_jogo

In [ ]:
t = cruzamentoPadrao(tabuleiroInincial(4),
                    tabuleiroInincial(4))
print(t)

In [ ]:
t1 = mutacaoPadrao(t, 1)
print(t1)

In [ ]:
t2 = cruzamentoPadrao(t, t1)
print(t2)

In [ ]:
def algoritmoGenetico(populacao, prob_mutacao, f_heuristica, f_cruzamento, f_mutacao, mutabilidade = 1, MAX = 5000):
    
    for iter in tqdm(range(MAX)):
        
        for i in range(0, len(populacao)):
        
            nova_populacao = []
            
            p_map = list(map(lambda x: (f_heuristica(x),x), populacao))

            p_min = sorted(p_map, key = lambda x: x[0])
            
            for _,x in p_min:
                
                #x = random.choice(p_min)[1]
                y = random.choice(p_min)[1]
            
                filho = f_cruzamento(x, y)
            
                aux = random.choices([True, False], weights= [prob_mutacao, 1-prob_mutacao])[0]

                if aux: filho = f_mutacao(filho, mutabilidade)
            
                nova_populacao.append(filho)
            
            sobreviventes = list(map(lambda x: x[1], filter(lambda x: x[0] == p_min[0][0], p_min)))
            populacao = [*nova_populacao, *sobreviventes]
        
        p_map = list(map(lambda x: (f_heuristica(x),x), nova_populacao))
        
        #print(p_map)
        
        p_min = sorted(p_map, key = lambda x: x[0])[0]
        
        if p_min[0] == 0: return p_min[1]
    
    p_map = list(map(lambda x: (f_heuristica(x),x), nova_populacao))
        
    return sorted(p_map, key = lambda x: x[0])[0][1]
        

In [ ]:
def tabuleiroRandomico(dimensao):
    jogo = Jogo(dimensao)
    
    l = range(dimensao)
    
    jogo.pos_damas = {i: (i, random.choice(l)) for i in range(dimensao)}
    
    return jogo

In [ ]:
pop = [tabuleiroRandomico(9) for x in range(100)]

In [ ]:
print(pop[0])

In [ ]:
teste = algoritmoGenetico(pop,
                 0.01,
                  heuristicaOtimizada,
                  cruzamentoPadrao,
                  mutacaoPadrao)

In [ ]:
print(teste)

In [ ]:
heuristicaOtimizada(teste)

In [60]:
jogo = tabuleiroInincial(10)

print(jogo)

┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┐
│ ♛ │   │   │   │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │ ♛ │   │   │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │ ♛ │   │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │ ♛ │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │ ♛ │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │ ♛ │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │ ♛ │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │   │ ♛ │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │   │   │ ♛ │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │   │   │   │ ♛ │
└───┴───┴───┴───┴───┴───┴───┴───┴───┴───┘



<h1 style="color:#fff;background-color:#000;padding:30px;margin:0;border:0;text-align:center">COLÔNIA DE FORMIGAS</h1>

In [12]:
def heuristicaIcaro(jogo):
    
    #peso do tabuleiro
    peso = 0

    #conjunto de pecas nao verificadas
    partes_ocupadas = list(jogo.pos_damas.keys())
    
    for i in range(len(partes_ocupadas)):
        
        for j in range(i+1, len(partes_ocupadas)):
            
            posi = np.array(jogo[partes_ocupadas[i]])
            
            posj = np.array(jogo[partes_ocupadas[j]])
            
            delta = abs(posi - posj)
            
            if delta[0] == delta[1] or posi[0] == posj[0] or posi[1] == posj[1]:
                peso += 1
        
    return peso

In [22]:

class ColoniaFormigas:
    def __init__(self, dimensao, quantidade_formigas):
        self.dimensao = dimensao
        self.quantidade_formigas = quantidade_formigas
        self.ro = 0.5 # (1 - ro) evaporação
        self.Q  = 1.0 # Quantidade de feromônio distribuído
        self.alfa = 1.0 # peso do feromônio
        self.beta = 1.0 # peso do peso da aresta
        self.formigas  = []
        self.feromonio = [[{'f':1, 'p':1} for i in range(dimensao)] for j in range(dimensao) ]
        self.calcula_feromonio()
        self.max = 1000
    
    def percorre(self):
        dimensao = self.dimensao
        jogo = Jogo(dimensao)
        linhas_possiveis = [i for i in range(dimensao)]
        p = np.array([1 for i in range(dimensao)])
        p = p/jogo.dimensao

        for coluna in range(dimensao):
            linha = int(choice(linhas_possiveis, 1, p=p)[0])
            linhas_possiveis.remove(linha)
            p = self.calcula_probabilidade(linha, linhas_possiveis)
            jogo[coluna] = (linha, coluna)

        self.formigas.append({'jogo':jogo, 'peso':heuristicaIcaro(jogo)})
    
    def calcula_probabilidade(self, linha, linhas_possiveis):
        p = []
        soma = 0.0
        for indice in linhas_possiveis:
            
            fero = self.feromonio[linha][indice]['f']
            peso = self.feromonio[linha][indice]['p']
            
            tudo = (fero**self.alfa)*(peso**self.beta)
            
            p.append(tudo)
            soma += (tudo)
        
        p = np.array([x for x in p])
        return p/soma
        
    def calcula_feromonio(self):
        dimensao = self.dimensao
       
        for i in range(dimensao):
            for j in range(dimensao):
                self.feromonio[i][j]['f'] = (1 - self.ro) * self.feromonio[i][j]['f']

                for formiga in self.formigas:
                    if (i, j) in formiga['jogo'].pos_damas.values():
                        self.feromonio[i][j]['f'] += self.Q/formiga['peso']

    def calcular(self):
        dimensao = self.dimensao
        
        for i in tqdm(range(1, self.max)):
            self.formigas = []
            threads = []
            
            for i in range(self.quantidade_formigas):
                t = Thread(target=self.percorre())
                t.start()
                threads.append(t)
                
            for t in threads:
                t.join()
            
            for formiga in self.formigas: 
                if formiga['peso'] == 0: return formiga

            self.calcula_feromonio()
        
        menor = self.formigas[0]
        for formiga in self.formigas:
            if formiga['peso'] < menor['peso']: menor = formiga
                
        return menor

In [24]:
col = ColoniaFormigas(15, 10)
col.max = 10000
a = col.calcular()
print(a['peso'])
print(a['jogo'])


7

┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┬───┐
│ ♛ │   │   │   │   │   │   │   │   │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │   │   │ ♛ │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │ ♛ │   │   │   │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │   │ ♛ │   │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │ ♛ │   │   │   │   │   │   │   │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │ ♛ │   │   │   │   │   │   │   │   │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │   │   │   │   │   │   │ ♛ │   │   │
├───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │   │   │   │   │   │   │   │ ♛ │   │
├───